In [2]:
import tsai.all
import datetime
import pandas as pd
import torch
import numpy as np
import wandb
from fastai.callback.wandb import WandbCallback
from fastai.callback.tracker import EarlyStoppingCallback, SaveModelCallback
import time
import math

In [3]:
%run ../shared_functions.py
%run ../my_shared_functions.py

<Figure size 640x480 with 0 Axes>

#### Preprocessing for first and second test (W&B testing on train test split, W&B Sweeps on train test split)

In [4]:
DIR_INPUT = '../../fraud-detection-handbook/simulated-data-transformed/data/'

BEGIN_DATE = "2018-06-11"
END_DATE = "2018-09-14"

print("Load  files")
%time transactions_df=read_from_files(DIR_INPUT, BEGIN_DATE, END_DATE)
print("{0} transactions loaded, containing {1} fraudulent transactions".format(len(transactions_df),transactions_df.TX_FRAUD.sum()))

output_feature="TX_FRAUD"

input_features=['TX_AMOUNT','TX_DURING_WEEKEND', 'TX_DURING_NIGHT', 'CUSTOMER_ID_NB_TX_1DAY_WINDOW',
       'CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW', 'CUSTOMER_ID_NB_TX_7DAY_WINDOW',
       'CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW', 'CUSTOMER_ID_NB_TX_30DAY_WINDOW',
       'CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW', 'TERMINAL_ID_NB_TX_1DAY_WINDOW',
       'TERMINAL_ID_RISK_1DAY_WINDOW', 'TERMINAL_ID_NB_TX_7DAY_WINDOW',
       'TERMINAL_ID_RISK_7DAY_WINDOW', 'TERMINAL_ID_NB_TX_30DAY_WINDOW',
       'TERMINAL_ID_RISK_30DAY_WINDOW']

Load  files
CPU times: total: 516 ms
Wall time: 492 ms
919767 transactions loaded, containing 8195 fraudulent transactions


In [5]:
BEGIN_DATE_CONFIG = "2018-07-25"
start_date_training = datetime.datetime.strptime(BEGIN_DATE_CONFIG, "%Y-%m-%d")
delta_train=7
delta_delay=7
delta_test=7


delta_valid = delta_test

start_date_training_with_valid = start_date_training+datetime.timedelta(days=-(delta_delay+delta_valid))

(train_df, valid_df)=get_train_test_set(transactions_df,start_date_training_with_valid,
                                       delta_train=delta_train,delta_delay=delta_delay,delta_test=delta_test)

# standardizing
(train_df, valid_df)=scaleData(train_df, valid_df,input_features)

SEQ_LEN = 5

def prepare_sequenced_X_y(df, seq_len, input_features, output_feature):
    x = torch.FloatTensor(df[input_features].values) # shape => [66928, 15] for train
    y = torch.FloatTensor(df[output_feature].values)
    features = torch.vstack([x, torch.zeros(x[0,:].shape)]) # shape => [66929, 15] for train
    df_ids_dates = pd.DataFrame({'CUSTOMER_ID':df['CUSTOMER_ID'].values,
            'TX_DATETIME':df['TX_DATETIME'].values})
    df_ids_dates["tmp_index"]  = np.arange(len(df_ids_dates))
    df_groupby_customer_id = df_ids_dates.groupby("CUSTOMER_ID")
    sequence_indices = pd.DataFrame(
        {
            "tx_{}".format(n): df_groupby_customer_id["tmp_index"].shift(SEQ_LEN - n - 1)
            for n in range(SEQ_LEN)
        }
    )

    sequences_ids = sequence_indices.fillna(len(features) - 1).values.astype(int) # shape => [66928, 5] for train

    x_sequenced = [features[sequences_ids[index], :].transpose(0, 1) for index in range(x.shape[0])]
    return torch.stack(x_sequenced), y # x shape => [66928, 15, 5] for train

In [6]:
SEED = 42

if torch.cuda.is_available():
    DEVICE = "cuda" 
else:
    DEVICE = "cpu"
print("Selected device is",DEVICE)

seed_everything(SEED)

x_train, y_train = prepare_sequenced_X_y(train_df, SEQ_LEN, input_features, output_feature)
x_valid, y_valid = prepare_sequenced_X_y(valid_df, SEQ_LEN, input_features, output_feature)

Selected device is cuda


In [7]:
X, y, splits = tsai.all.combine_split_data([x_train.numpy(), x_valid.numpy()], [y_train.numpy(), y_valid.numpy()])

#### W&B testing on simple train test split (GRU default example)

In [14]:
config = dict(
    dataset_id = 'fraud-detection-handbook-transformed',
    validation = 'train test split',
    seed = SEED,
    begin_date = BEGIN_DATE_CONFIG,
    delta_train = delta_train,
    delta_delay = delta_delay,
    delta_test = delta_test,
    batch_size=64,
    num_workers=0,
    seq_len=SEQ_LEN,
    hidden_size = 100,
    n_layers = 1,
    rnn_dropout=0,
    bidirectional=False,
    fc_dropout=0,
    optimizer='adam',
    lr=0.0001,
    early_stopping=True,
    early_stopping_patience=5,
    max_epochs=100,
    scale=True,
    criterion='bce'
)
wandb.init(project="mgr-anomaly-tsxai-project", config=config, tags=['gru', 'tsai', 'imbalance-not-considered'])
config = wandb.config

In [15]:
dsets = tsai.all.TSDatasets(X, y, splits=splits, inplace=True)
dls = tsai.all.TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=config.batch_size, num_workers=config.num_workers, drop_last=True, device=DEVICE)

In [16]:
gru = tsai.all.GRU(dls.vars, dls.c)
gru

GRU(
  (rnn): GRU(15, 100, batch_first=True)
  (dropout): Identity()
  (fc): Linear(in_features=100, out_features=1, bias=True)
)

From https://github.com/timeseriesAI/tsai/blob/main/tutorial_nbs/12_Experiment_tracking_with_W%26B.ipynb November 2022:

"There's currently a small bug in the integration between wandb and tsai that doesn't allow to log_preds. This can be used to show predictions in W&B. We recommend setting log_preds=False."

In [17]:
criterion = tsai.all.BCEWithLogitsLossFlat()
optimizer = tsai.all.wrap_optimizer(torch.optim.Adam, **{'lr': config.lr})
learn = tsai.all.ts_learner(
    dls,
    gru,
    loss_func=criterion,
    opt_func=optimizer,
    device=torch.device('cuda'),
    lr=config.lr,
    metrics=[],
    model_dir='GRU',
    cbs=[
        WandbCallback(
            log='all',
            log_model=True,
            seed=config.seed),
        EarlyStoppingCallback(
            patience=config.early_stopping_patience
        ),
        SaveModelCallback(
            fname='gru'
        )])

start_time=time.time()
learn.fit(config.max_epochs)
training_execution_time=time.time()-start_time

# learn.fit_one_cycle(config.max_epochs)

WandbCallback was not able to prepare a DataLoader for logging prediction samples -> 'NoneType' object is not iterable


epoch,train_loss,valid_loss,time
0,0.032060,0.027005,00:09
1,0.033919,0.024424,00:09
2,0.030790,0.023034,00:09
3,0.030508,0.022038,00:09
4,0.020360,0.020844,00:09
5,0.018333,0.020582,00:09
6,0.027894,0.020526,00:09
7,0.023275,0.019861,00:09
8,0.024370,0.019721,00:09
9,0.025996,0.019817,00:09


Better model found at epoch 0 with valid_loss value: 0.027004871517419815.
Better model found at epoch 1 with valid_loss value: 0.024424413219094276.
Better model found at epoch 2 with valid_loss value: 0.023034116253256798.
Better model found at epoch 3 with valid_loss value: 0.022038085386157036.
Better model found at epoch 4 with valid_loss value: 0.020844215527176857.
Better model found at epoch 5 with valid_loss value: 0.02058219164609909.
Better model found at epoch 6 with valid_loss value: 0.02052551507949829.
Better model found at epoch 7 with valid_loss value: 0.019861457869410515.
Better model found at epoch 8 with valid_loss value: 0.019720803946256638.
Better model found at epoch 10 with valid_loss value: 0.01960168220102787.
Better model found at epoch 11 with valid_loss value: 0.019488301128149033.
Better model found at epoch 12 with valid_loss value: 0.019459718838334084.
Better model found at epoch 13 with valid_loss value: 0.019331851974129677.
Better model found at ep

In [18]:
valid_dl = learn.dls.valid
start_time=time.time()
valid_probas, valid_targets, valid_preds = learn.get_preds(dl=valid_dl, with_decoded=True)
prediction_execution_time=time.time()-start_time
predictions_df = valid_df
predictions_df['predictions'] = valid_preds
    
performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])
performance_df

,AUC ROC,Average precision,F1 score,Card Precision@100
0,0.772,0.467,0.664,0.223


In [19]:
wandb.log({'Training execution time': training_execution_time})
wandb.log({'Prediction execution time': prediction_execution_time})
wandb.log({'AUC ROC': performance_df.loc[0,'AUC ROC']})
wandb.log({'Average precision': performance_df.loc[0,'Average precision']})
wandb.log({'F1 score': performance_df.loc[0,'F1 score']})
wandb.log({'Card Precision@100': performance_df.loc[0,'Card Precision@100']})
wandb.finish()

AUC ROC,▁
Average precision,▁
Card Precision@100,▁
F1 score,▁
Prediction execution time,▁
Training execution time,▁
amsgrad_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
capturable_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
differentiable_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


#### W&B Sweeps on simple train test split

https://www.youtube.com/watch?v=9zrmUIlScdY

https://colab.research.google.com/github/wandb/examples/blob/master/colabs/pytorch/Organizing_Hyperparameter_Sweeps_in_PyTorch_with_W%26B.ipynb

In [22]:
sweep_config = {
                'method': 'random', 
                'metric': {
                    'goal': 'minimize',
                    'name': 'val_loss'
                    },
                'parameters': {
                    'batch_size': {
                        # integers between 32 and 256 with equally evenly-distributed logarithms
                        # 'distribution': 'q_log_uniform',
                        # 'max': math.log(256),
                        # 'min': math.log(32),
                        # 'q': 1
                        'values' : [64, 128]
                    },
                    'max_epochs': {
                        'value': 100
                        },
                    'num_workers': {
                        'value' : 0
                    },
                    'lr': {
                        # flat distribution between 0 and 0.1
                        # 'distribution': 'uniform',
                        # 'max': 0.1,
                        # 'min': 0},
                        'values' : [0.0001, 0.001]
                    },
                    'seed': {
                        'value' : 42
                    },
                    'early_stopping_patience': {
                        'value' : 5
                    },
                }
 }

sweep_id = wandb.sweep(sweep_config, project="mgr-anomaly-tsxai-project")

Create sweep with ID: 9dcp5xcj
Sweep URL: https://wandb.ai/mgr-anomaly-tsxai/mgr-anomaly-tsxai-project/sweeps/9dcp5xcj


In [23]:
dsets = tsai.all.TSDatasets(X, y, splits=splits, inplace=True)

In [24]:
def train(config=None):
    with wandb.init(project="mgr-anomaly-tsxai-project", config=config, tags=['gru', 'tsai', 'imbalance-not-considered', 'sweeps']):
        config = wandb.config
        dls = tsai.all.TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=config.batch_size, num_workers=config.num_workers, drop_last=True, device=DEVICE)
        gru = tsai.all.GRU(dls.vars, dls.c)
        criterion = tsai.all.BCEWithLogitsLossFlat()
        optimizer = tsai.all.wrap_optimizer(torch.optim.Adam, **{'lr': config.lr})
        learn = tsai.all.ts_learner(
            dls,
            gru,
            loss_func=criterion,
            opt_func=optimizer,
            device=torch.device('cuda'),
            lr=config.lr,
            metrics=[],
            model_dir='GRU',
            cbs=[
                WandbCallback(
                    log='all',
                    log_model=True,
                    seed=config.seed),
                EarlyStoppingCallback(
                    patience=config.early_stopping_patience
                ),
                SaveModelCallback(
                    fname='gru'
                )])

        start_time=time.time()
        learn.fit(config.max_epochs)
        training_execution_time=time.time()-start_time

        valid_dl = learn.dls.valid
        start_time=time.time()
        valid_probas, valid_targets, valid_preds = learn.get_preds(dl=valid_dl, with_decoded=True)
        prediction_execution_time=time.time()-start_time
        predictions_df = valid_df
        predictions_df['predictions'] = valid_preds
        performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])

        wandb.log({'Training execution time': training_execution_time})
        wandb.log({'Prediction execution time': prediction_execution_time})
        wandb.log({'AUC ROC': performance_df.loc[0,'AUC ROC']})
        wandb.log({'Average precision': performance_df.loc[0,'Average precision']})
        wandb.log({'F1 score': performance_df.loc[0,'F1 score']})
        wandb.log({'Card Precision@100': performance_df.loc[0,'Card Precision@100']})

In [25]:
wandb.agent(sweep_id, train, count=4)

wandb: Agent Starting Run: z6z2p0yh with config:
wandb: 	batch_size: 64
wandb: 	early_stopping_patience: 5
wandb: 	lr: 0.0001
wandb: 	max_epochs: 100
wandb: 	num_workers: 0
wandb: 	seed: 42
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


WandbCallback was not able to prepare a DataLoader for logging prediction samples -> 'NoneType' object is not iterable


epoch,train_loss,valid_loss,time
0,0.036574,0.027188,00:11
1,0.030581,0.024536,00:11
2,0.031362,0.023153,00:11
3,0.032207,0.022058,00:11
4,0.029958,0.021114,00:11
5,0.023016,0.020511,00:11
6,0.021363,0.020059,00:11
7,0.020540,0.019855,00:11
8,0.020230,0.019809,00:11
9,0.021125,0.019736,00:12


Better model found at epoch 0 with valid_loss value: 0.027187876403331757.
Better model found at epoch 1 with valid_loss value: 0.024536387994885445.
Better model found at epoch 2 with valid_loss value: 0.02315320260822773.
Better model found at epoch 3 with valid_loss value: 0.022057687863707542.
Better model found at epoch 4 with valid_loss value: 0.02111424319446087.
Better model found at epoch 5 with valid_loss value: 0.020511161535978317.
Better model found at epoch 6 with valid_loss value: 0.02005911059677601.
Better model found at epoch 7 with valid_loss value: 0.019855299964547157.
Better model found at epoch 8 with valid_loss value: 0.019809087738394737.
Better model found at epoch 9 with valid_loss value: 0.019736459478735924.
Better model found at epoch 10 with valid_loss value: 0.01956108771264553.
Better model found at epoch 11 with valid_loss value: 0.01945381984114647.
Better model found at epoch 14 with valid_loss value: 0.019298575818538666.
Better model found at epoch

AUC ROC,▁
Average precision,▁
Card Precision@100,▁
F1 score,▁
Prediction execution time,▁
Training execution time,▁
amsgrad_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
capturable_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
differentiable_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


...

wandb: Agent Starting Run: m94gaic9 with config:
wandb: 	batch_size: 64
wandb: 	early_stopping_patience: 5
wandb: 	lr: 0.001
wandb: 	max_epochs: 100
wandb: 	num_workers: 0
wandb: 	seed: 42
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


WandbCallback was not able to prepare a DataLoader for logging prediction samples -> 'NoneType' object is not iterable


epoch,train_loss,valid_loss,time
0,0.023949,0.019916,00:11
1,0.021559,0.019595,00:11
2,0.024822,0.019044,00:11
3,0.023151,0.019116,00:11
4,0.023873,0.018819,00:11
5,0.017503,0.020422,00:11
6,0.019777,0.019098,00:11
7,0.016447,0.017897,00:11
8,0.015515,0.017958,00:11
9,0.020293,0.019217,00:11


Better model found at epoch 0 with valid_loss value: 0.01991594396531582.
Better model found at epoch 1 with valid_loss value: 0.019595175981521606.
Better model found at epoch 2 with valid_loss value: 0.019043751060962677.
Better model found at epoch 4 with valid_loss value: 0.018818840384483337.
Better model found at epoch 7 with valid_loss value: 0.017897048965096474.
No improvement since epoch 7: early stopping


AUC ROC,▁
Average precision,▁
Card Precision@100,▁
F1 score,▁
Prediction execution time,▁
Training execution time,▁
amsgrad_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
capturable_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
differentiable_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


#### W&B Sweeps on prequential validation

In [ ]:
# Sweeps on TSAI train test split
#
# load transcations df
# get train test set
# scale data
# prepare x_train, y_train, x_val, y_val
# combine split data (tsai)
# define sweep config
# initialize a hp sweep (wand.sweep())
# create TSDatasets (tsai)
# create train(config) function:
#   wandb.init()
#   create data loaders
#   create model instance
#   train model
#   calculate performance
# run sweeps (wandb.agent())


# Grid hp search on PyTorch prequential split
#
# load transactions df
# run prequential_grid_search(
#   transactions_df,
#   neural net instance,
#   hyperparameters dict,
#   start_date_training_with_valid - fixed datetime value,
#   n_folds,
#   delta_train, delta_delay, delta_valid,
#   list of performance metrics
# )
#
# inside prequential_grid_search:
#   pipeline creation: scaler + neural net
#   prequentialSplit function -> returns prequential split indices
#       for each fold (for loop): get train test set based on start date training, delta train, delta delay and delta test
#       return list of indices for each fold
#   grid search: create instance, get X and Y from transcations_df, run grid search
#   calculate performance

In [4]:
DIR_INPUT = '../../fraud-detection-handbook/simulated-data-transformed/data/'

BEGIN_DATE = "2018-06-11"
END_DATE = "2018-09-14"

print("Load  files")
%time transactions_df=read_from_files(DIR_INPUT, BEGIN_DATE, END_DATE)
print("{0} transactions loaded, containing {1} fraudulent transactions".format(len(transactions_df),transactions_df.TX_FRAUD.sum()))

output_feature="TX_FRAUD"

input_features=['TX_AMOUNT','TX_DURING_WEEKEND', 'TX_DURING_NIGHT', 'CUSTOMER_ID_NB_TX_1DAY_WINDOW',
       'CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW', 'CUSTOMER_ID_NB_TX_7DAY_WINDOW',
       'CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW', 'CUSTOMER_ID_NB_TX_30DAY_WINDOW',
       'CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW', 'TERMINAL_ID_NB_TX_1DAY_WINDOW',
       'TERMINAL_ID_RISK_1DAY_WINDOW', 'TERMINAL_ID_NB_TX_7DAY_WINDOW',
       'TERMINAL_ID_RISK_7DAY_WINDOW', 'TERMINAL_ID_NB_TX_30DAY_WINDOW',
       'TERMINAL_ID_RISK_30DAY_WINDOW']

Load  files
CPU times: total: 516 ms
Wall time: 506 ms
919767 transactions loaded, containing 8195 fraudulent transactions


In [5]:
start_date_training = datetime.datetime.strptime("2018-07-25", "%Y-%m-%d")
delta_train=7
delta_delay=7
delta_test=7
delta_valid = delta_test

n_folds=4

start_date_training_for_valid = start_date_training+datetime.timedelta(days=-(delta_delay+delta_valid))
start_date_training_for_test = start_date_training+datetime.timedelta(days=(n_folds-1)*delta_test)
delta_assessment = delta_valid

In [6]:
SEED = 42
seed_everything(SEED)

prequential_split = []
        
for fold in range(n_folds):
    start_date_training_fold = start_date_training-datetime.timedelta(days=fold*delta_assessment)
    
    (train_df, test_df)=get_train_test_set(transactions_df,
                                            start_date_training=start_date_training_fold,
                                            delta_train=delta_train,delta_delay=delta_delay,delta_test=delta_assessment)
    
    prequential_split.append((train_df, test_df))

In [7]:
SEQ_LEN = 5

def prepare_sequenced_X_y(df, seq_len, input_features, output_feature):
    x = torch.FloatTensor(df[input_features].values) # shape => [66928, 15] for train
    y = torch.FloatTensor(df[output_feature].values)
    features = torch.vstack([x, torch.zeros(x[0,:].shape)]) # shape => [66929, 15] for train
    df_ids_dates = pd.DataFrame({'CUSTOMER_ID':df['CUSTOMER_ID'].values,
            'TX_DATETIME':df['TX_DATETIME'].values})
    df_ids_dates["tmp_index"]  = np.arange(len(df_ids_dates))
    df_groupby_customer_id = df_ids_dates.groupby("CUSTOMER_ID")
    sequence_indices = pd.DataFrame(
        {
            "tx_{}".format(n): df_groupby_customer_id["tmp_index"].shift(SEQ_LEN - n - 1)
            for n in range(SEQ_LEN)
        }
    )

    sequences_ids = sequence_indices.fillna(len(features) - 1).values.astype(int) # shape => [66928, 5] for train

    x_sequenced = [features[sequences_ids[index], :].transpose(0, 1) for index in range(x.shape[0])]
    return torch.stack(x_sequenced), y # x shape => [66928, 15, 5] for train

In [8]:
if torch.cuda.is_available():
    DEVICE = "cuda" 
else:
    DEVICE = "cpu"
print("Selected device is",DEVICE)

Selected device is cuda


In [9]:
sweep_config = {
                'method': 'random', 
                'metric': {
                    'goal': 'minimize',
                    'name': 'val_loss'
                    },
                'parameters': {
                    'batch_size': {
                        'values' : [64, 128]
                    },
                    'max_epochs': {
                        'values': [20, 40]
                        },
                    'num_workers': {
                        'value' : 0
                    },
                    'lr': {
                        'value' : 0.001
                    },
                    'seed': {
                        'value' : SEED
                    }
                }
 }

In [10]:
sweep_id = wandb.sweep(sweep_config, project="mgr-anomaly-tsxai-project")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: ps17q4yt
Sweep URL: https://wandb.ai/mgr-anomaly-tsxai/mgr-anomaly-tsxai-project/sweeps/ps17q4yt


In [11]:
def train(config=None):
    with wandb.init(project="mgr-anomaly-tsxai-project", config=config, tags=['gru', 'tsai', 'imbalance-not-considered', 'sweeps']):
        config = wandb.config
        training_execution_times = []
        prediction_execution_times = []
        aucs = []
        average_precisions = []
        f1_scores = []
        card_precisions = []
        for i in range(n_folds):
            train_df = prequential_split[i][0]
            valid_df = prequential_split[i][1]
            (train_df, valid_df)=scaleData(train_df, valid_df,input_features)
            x_train, y_train = prepare_sequenced_X_y(train_df, SEQ_LEN, input_features, output_feature)
            x_valid, y_valid = prepare_sequenced_X_y(valid_df, SEQ_LEN, input_features, output_feature)
            X, y, tsai_splits = tsai.all.combine_split_data([x_train.numpy(), x_valid.numpy()], [y_train.numpy(), y_valid.numpy()])
            dsets = tsai.all.TSDatasets(X, y, splits=tsai_splits, inplace=True)
            dls = tsai.all.TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=config.batch_size, num_workers=config.num_workers, drop_last=True, device=DEVICE)
            gru = tsai.all.GRU(dls.vars, dls.c)
            criterion = tsai.all.BCEWithLogitsLossFlat()
            optimizer = tsai.all.wrap_optimizer(torch.optim.Adam, **{'lr': config.lr})
            learn = tsai.all.ts_learner(
                dls,
                gru,
                loss_func=criterion,
                opt_func=optimizer,
                device=torch.device('cuda'),
                lr=config.lr,
                metrics=[],
                model_dir='GRU',
                # no early stopping for multiple folds validation!
                cbs=[
                    WandbCallback(
                        log=None,
                        log_model=False,
                        seed=config.seed)])

            start_time=time.time()
            learn.fit(config.max_epochs)
            training_execution_times.append(time.time()-start_time)

            valid_dl = learn.dls.valid
            start_time=time.time()
            valid_probas, valid_targets, valid_preds = learn.get_preds(dl=valid_dl, with_decoded=True)
            prediction_execution_times.append(time.time()-start_time)
            predictions_df = valid_df
            predictions_df['predictions'] = valid_preds
            performance_df = performance_assessment_f1_included(predictions_df, top_k_list=[100])
            aucs.append(performance_df.loc[0,'AUC ROC'])
            average_precisions.append(performance_df.loc[0,'Average precision'])
            f1_scores.append(performance_df.loc[0,'F1 score'])
            card_precisions.append(performance_df.loc[0,'Card Precision@100'])

        wandb.log({'Training execution time': np.sum(training_execution_times) / n_folds})
        wandb.log({'Prediction execution time': np.sum(prediction_execution_times) / n_folds})
        wandb.log({'AUC ROC': np.sum(aucs) / n_folds})
        wandb.log({'Average precision': np.sum(average_precisions) / n_folds})
        wandb.log({'F1 score': np.sum(f1_scores) / n_folds})
        wandb.log({'Card Precision@100': np.sum(card_precisions) / n_folds})

In [12]:
wandb.agent(sweep_id, train, count=2)

wandb: Agent Starting Run: iosr949p with config:
wandb: 	batch_size: 64
wandb: 	lr: 0.001
wandb: 	max_epochs: 40
wandb: 	num_workers: 0
wandb: 	seed: 42
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: chamera (mgr-anomaly-tsxai). Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


WandbCallback was not able to prepare a DataLoader for logging prediction samples -> 'NoneType' object is not iterable


epoch,train_loss,valid_loss,time
0,0.020459,0.020353,00:09
1,0.024615,0.019615,00:09
2,0.016570,0.019771,00:09
3,0.019064,0.019145,00:09
4,0.019634,0.019599,00:09
5,0.017632,0.018905,00:09
6,0.020449,0.018184,00:09
7,0.012932,0.018725,00:09
8,0.015476,0.018423,00:09
9,0.016105,0.018384,00:09


WandbCallback was not able to prepare a DataLoader for logging prediction samples -> 'NoneType' object is not iterable


epoch,train_loss,valid_loss,time
0,0.022214,0.021960,00:09
1,0.026649,0.020738,00:09
2,0.014774,0.021597,00:09
3,0.025191,0.020126,00:09
4,0.017033,0.019485,00:09
5,0.015650,0.019220,00:09
6,0.013567,0.020012,00:09
7,0.018427,0.018926,00:09
8,0.013281,0.019036,00:10
9,0.019604,0.018996,00:09


...

WandbCallback was not able to prepare a DataLoader for logging prediction samples -> 'NoneType' object is not iterable


epoch,train_loss,valid_loss,time
0,0.022119,0.020985,00:05
1,0.020633,0.019932,00:05
2,0.025188,0.019330,00:05
3,0.024620,0.020232,00:05
4,0.022379,0.019377,00:05
5,0.023406,0.018580,00:05
6,0.017766,0.018461,00:05
7,0.022405,0.019119,00:05
8,0.019990,0.017999,00:05
9,0.019204,0.017771,00:05


WandbCallback was not able to prepare a DataLoader for logging prediction samples -> 'NoneType' object is not iterable


epoch,train_loss,valid_loss,time
0,0.027026,0.020064,00:04
1,0.023216,0.017756,00:05
2,0.020657,0.017390,00:04
3,0.020330,0.017098,00:04
4,0.019313,0.016561,00:05
5,0.015613,0.016237,00:05
6,0.017935,0.016218,00:05
7,0.017838,0.016409,00:04
8,0.019137,0.015813,00:05
9,0.015469,0.016403,00:04


AUC ROC,▁
Average precision,▁
Card Precision@100,▁
F1 score,▁
Prediction execution time,▁
Training execution time,▁
amsgrad_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
capturable_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
differentiable_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
